In [1]:
from brainlit.utils import upload, session, upload_benchmarking
from pathlib import Path
import napari

c:\users\shrey\anaconda3\envs\ndd-windows\lib\site-packages\python_jsonschema_objects\__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


In [2]:
#to delete
import numpy as np
import tifffile as tf
import tqdm
from brainlit.utils.util import (
    tqdm_joblib,
    check_type,
    check_iterable_type,
    check_size,
    check_precomputed,
    check_binary_path,
)
from brainlit.utils.swc import swc2skeleton


## Uploading Brain Images from data in the Octree format.
This notebook demonstrates uploading the 2 lowest-resolution brain volumes, as well as a `.swc` segment file.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `p` is the prefix string. `file://` indicates a filepath, while `s3://` or `gc://` indicate URLs.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, filepaths).
 - `num_res` denotes the number of resolutions to upload.

In [3]:
#source = Path("C:/Users/shrey/Downloads/Images").as_posix()
source = Path("C:/Users/shrey/Downloads/benchmarking_datasets").as_posix()
source_segments = Path("C:/Users/shrey/Downloads/benchmarking_datasets/Manual-GT").as_posix()
dest = "s3://open-neurodata/benchmarking_data/Images"
dest_segments = "s3://open-neurodata/benchmarking_data/Manual-GT"
num_res = 1

In [38]:
#source = (Path().resolve().parents[2] / "data" / "data_octree").as_posix()
#dest = (Path().resolve().parents[2] / "data" / "upload").as_uri()
#dest_segments = (Path().resolve().parents[2] / "data" / "upload_segments").as_uri()
#dest_annotation = (Path().resolve().parents[2] / "data" / "upload_annotation").as_uri()
#num_res = 2

### Uploading .tif files

In [34]:
files_ordered, paths_bin, vox_size, tiff_dims, origin = upload_benchmarking.get_volume_info(source, num_res)

IndexError: list index out of range

In [24]:
vols = upload_benchmarking.create_cloud_volume(dest, tiff_dims, vox_size, num_res)

In [4]:
upload_benchmarking.upload_volumes(source, dest, num_res)

Preparing files.
Starting upload.




timed out on a chunk on layer index 0. moving on to the next step of pipeline


In [5]:
upload_benchmarking.upload_volumes(source, dest, num_res, continue_upload = (0, 19))

Preparing files.
Starting upload.




        Input image shape does not match slice shape.

        Image Shape: [330, 312, 100, 1]  
        Slice Shape: [330, 330, 100, 1]
      
timed out on a chunk on layer index 0. moving on to the next step of pipeline


In [6]:
upload_benchmarking.upload_volumes(source, dest, num_res, continue_upload = (0, 35))

Preparing files.
Starting upload.




timed out on a chunk on layer index 0. moving on to the next step of pipeline


In [7]:
upload_benchmarking.upload_volumes(source, dest, num_res, continue_upload = (0, 46))

Preparing files.
Starting upload.




Finished layer index 0, took 30.188754320144653 seconds


### Uploading .swc files

In [4]:
skeletons, segids = upload_benchmarking.create_skel_segids(source_segments, origin = (0,0,0))

In [5]:
upload_benchmarking.upload_segments(source, dest_segments, num_res)

Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 10.33it/s]


Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 13.88it/s]


Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 14.97it/s]


Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 11.37it/s]


Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 13.01it/s]


Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 14.97it/s]


Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 14.25it/s]


## 2) Upload the image data.

In [ ]:
upload.upload_volumes(source, dest, num_res)

## 3) Upload the segmentation data.

In [ ]:
upload.upload_segments(source, dest_segments, num_res)

## Download the data with NeuroglancerSession and generate labels.

In [ ]:
%%capture
sess = session.NeuroglancerSession(url=dest, url_segments=dest_segments, mip=0)  # create session object object
img, bounds, vertices = sess.pull_vertex_list(2, range(250, 350), expand=True)  # get image containing some data
labels = sess.create_tubes(2, bounds, radius=1)  # generate labels via tube segmentation

## 4) Visualize the data with napari

In [ ]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_labels(labels)